# Notebook 6.1: `np.random` and `np.histogram`

### Required software
Before proceeding be sure to install the packages below using conda, and then restart your notebook. Here we will mostly focus on numpy, but will use some code from the toyplot and scipy libraries as well.

In [ ]:
# conda install toyplot -c conda-forge
# conda install numpy -c conda-forge
# conda install scipy -c conda-forge

In [ ]:
import numpy as np
import scipy
import toyplot

### Numpy random

The `numpy.random` package is one of the most useful scientific packages you are likely to use. It will feel familiar because it has many of the same features as the `random` package from the Python standard library, but the numpy version is much more expansive and also much faster. 


### Generative data
Random sampling functions are fundamental to scientific programming since they provide a simple way for you to generate null data under a known model for inputing into your functions to test that they return the expected result (e.g., infer the correct parameters of the model). For example, if we were writing a statistical package to perform a linear regression to fit the model `y = ax + b + e`, where x is the slope, a is the intercept, and e is an error term. We can generate data under this model by sampling random values for x, and generating values for y by transforming them with a known a, b and e values. Here is an example:


In [ ]:
# slope (a parameter our model would try to infer)
a = 3.0

In [ ]:
# intercept (a parameter our model would try to infer)
b = 5.0

In [ ]:
# random noise (a parameter our model would try to infer)
e = np.random.normal(loc=0.0, scale=1.0, size=1000)

In [ ]:
# 1000 random x values (data)
x = np.random.normal(loc=0.0, scale=1.0, size=1000)

In [ ]:
# 1000 y values (generative data: made under our known model)
y = a * x + b + e

In [ ]:
# plot the data 
toyplot.scatterplot(
    x, y,
    size=6,
    opacity=0.2,
    height=250,
    width=300,
);

We can now fit a simple linear regression model. There are many ways to do this in Python. Here we use a function from the scipy package to just estimate a and b. We can see that it works well, the slope is very close to 3 and the intercept is very close to 5. The r-squared value is nearly 1, showing a very strong correlation. If we were to increase the e value above the r-value will decrease but because the error is randomly distributed around 0 it should not bias the slope and intercept parameter inference until it becomes very large.  

In [ ]:
# fit a simple linear regression model
import scipy.stats
scipy.stats.linregress(x, y)

### Random sampling
The example above introduces many things all at once. So let's break it down. How does random sampling work? This is actually a very complex and important topic in computing. It relies on a *random number generator* that ensures that each draw is statistically independent of the last. We can take advantage of this concept to provide a `seed` to the random generator such that it starts from the same point every time. The results are then random, but they are also repeatable. 

In the example above we sampled random values from a 'normal' distribution. You are likely familiar with this type of statistical distribution, it is the typical bell-curve type distribution. We can describe a normal distribution simply by providing a mean and standard deviation as parameters of the distribution. In the numpy function these are entered as the `loc` and `scale` parameters to `np.random.normal`. 

In addition to the normal distribution there are dozens of other statistical distributions that can be explored in numpy (as we'll learn later, the scipy library also has many additional ways of working with statistical distributions). We'll explore some of the numpy random sampling functions in this notebook.



In [ ]:
# example: get 10 random integers between 0 and 255
np.random.randint(0, 255, 10)

In [ ]:
# example: get 10 nucleotide bases from 'ACGT'
np.random.choice(list("ACGT"), 10)

# Masking 

You've already learned about masking. This is a way of filtering numpy arrays to only select certain elements by providing a boolean array. A simple example is below:

In [ ]:
# any boolean array can serve as a mask
mask = np.array([True, False, True, False])

# apply mask to an integer array will only show values with True
intarr = np.array([2, 3, 4, 5])
intarr[mask]

## `random.binomial` to randomly mask


Masking is an effective way to select only a subset of values in an array. This can be used to subsample randomly, or to filter values that meet only a certain criterion. Below are several ways to create a boolean mask to randomly sample values from an array efficiently. 

In [ ]:
# an array of 1000 sequential ints
arr = np.arange(1000)

#### random binomial trials
Binomial sampling can be thought of like a coin flip, but where you can assign the probability to each outcome like a weighted coin. Below we run 1000 trials (size) of individual coin flips (n=1) where the probability of an outcome being True (e.g., flipping heads) is 0.1 (p=0.1). This will return an array of booleans binary integers (e.g., `array([0, 0, 1, 1])`) which we will then convert to a boolean type using the `astype()` call. 

In [ ]:
# 1000 trials where each has success rate of 0.1
mask = np.random.binomial(n=1, p=0.1, size=1000).astype(bool)

# show the first 50 results
mask[:50]

#### masking with a boolean array
A boolean array can be used to subselect from another array by selecting only the elements of value `True` in the boolean array. Remember, True is a special keyword in Python, and it is equivalent to the value 1, which is why we were able to convert the 1's and 0's above into True's and False's so easily. Applying the mask from above that selected elements with a probability of 0.1 we see that it reduces the array of 1000 ordered integers into a smaller array of around 100 values. 

In [ ]:
# use boolean array to mask (select only element that are True)
arr[mask]

## Use `random.choice` for randomizing


Similar to above, instead of selecting True or False for every cell sometimes we may want to randomly sample values from an array while dictating the exact number that we will get in the end. This can be done with `random.choice`, and has a lot of potential uses in biological programming. One example is in the statistical method called bootstrap resampling. What `random.choice` is actually doing here is sampling random integers and then selecting the index of those elements in the array. I'll show a quick example of this first:

In [ ]:
# masking (subselecting) by indexing
index = np.array([0, 0, 3, 3])
data = np.array(['a', 'b', 'c', 'd'])

# select the indices from data
data[index]

#### Bootstrap resampling (e.g., bootstrapping)
Bootstrapping is a *non-parametric* method for testing the reliablility of a measurement by testing how representative an observed statistic is compared to a random re-sampling of the data points from which it was calculated. It provides a way of examining the variance in a statistic without needing to collect an entirely new data set, nor assuming that the data are distributed according to a standard statistical distribution, like a normal distribution. Instead, we just re-test the same data set by resampling it. Another way to think about it is that it is examining whether there are few outlier data points that might be driving our results, since when you resample data points you expect that the outliers may sometimes be left out and thus the calculated statistic may be very different. Let's try it out.


In [ ]:
# sample a random distribution of integers uniformly between 0 and 255 
data = np.random.uniform(0, 255, 1000)

# calculate a statistic on the observed data
dmean = data.mean()

# run one bootstrap replicate (sample w/o replacement to the same size as original)
boot0 = np.random.choice(data, size=data.size)

# print observed and single bootstrap (they're pretty similar)
print(data.mean(), boot0.mean())

#### plot bootstrap distribution and observed data point
As you can see our observed statistic falls right at the mean of our bootstrap distribution, thus we can say that our results are likely not skewed by large outliers, yet there is also a fair bit of variation around the mean so we now have a better estimate of uncertainty. (Don't worry about learning the code for plotting for now, we will cover this in a later tutorial.)

In [ ]:
# run 1000 boots using list-comprehension in an array
boots = np.array([np.random.choice(data, size=data.size).mean() for i in range(1000)])

In [ ]:
# plot bootstrap distribution of means
c, a, m = toyplot.bars(
    np.histogram(boots, bins=25), 
    opacity=0.5, 
    height=200, 
    width=400,
);

# add a vertical line at the True mean
a.vlines(
    data.mean(), 
    style={"stroke": "orange", "stroke-width": 4, "stroke-dasharray": "5,2"},
);

### Sampling from statistical distributions

For many statistical tests we are interested in comparing observed data to a known statistical distribution, or simulating data under a known statistical distribution to test whether observed data fit to some expected modeled outcome. The binomial distribution that we saw above is one such type of *parametric* model, where we provide a parameter (p; the probability of success in a trial) and simulate random runs under that model. Below we'll try out a few other common models used in biological programming. 

#### The uniform distribution 
The uniform distribution samples numbers with equal frequency within a set range of values (defined by `low` and `high`). This is similar to the `randint` function above, but in this case a `float` is returned, thus it is sampling randomly along all values within and between integers in the selected range. We are saying that all values in this range are equally likely to be sampled. 

In [ ]:
# sampling from a Uniform distribution
np.random.uniform(low=0, high=255, size=10)

#### The normal distribution 
This is the standard bell curve, the result of sampling from a distribution with a mean value and some variance around that mean. The normal distribution is thus parameterized with two values, a mean (`loc`) and a standard deviation (`scale`). 

In [ ]:
# sample from a Normal distribution
np.random.normal(loc=0, scale=2, size=10)

## Histograms
A histogram is a way of *binning* values that are within some range of each other into a discrete category, and is typically used as a way for visualizing large data sets. In your reading histograms were created using the `matplotlib` library, which internally calls the function `np.histogram` to bin values. (I think matplotlib is ugly and prefer the library `toyplot` so we will often use this instead.) When we call `np.histogram` on an array of values it returns two values (or a single tuple with two values) that hold the value of each bin as well as the edges of each bin. Pass these arrays to `toyplot.bars` to plot a histogram like below. Here I add two additional arguments to `np.histogram` to set the number of bins to 20, and to return the values as a frequency (`density`) as opposed to a count of the number of values in each bin. 



In [ ]:
# randomly sample values
arr = np.random.uniform(low=0, high=10, size=100000)

# bin values: returns an array with heights and one with bin edges.
hist, edges = np.histogram(arr, bins=20, density=True)

# pass bin data to bars
toyplot.bars(
    (hist, edges), 
    height=200, 
    width=400, 
    label="Uniform distributed random values",
);

In [ ]:
# randomly sample values
arr = np.random.normal(loc=0, scale=2, size=100000)

# bin values
hist, edges = np.histogram(arr, bins=20, density=True)

# draw with bars
toyplot.bars((hist, edges), height=200, width=400, label="Normal distributed random values");

#### Exponential distribution

The exponential distribution is the average *waiting time* between events that occur independently and with a fixed probability (called a Poisson process). For example, we could ask: if the mutation rate of an organism is 1e-8 then what is the average waiting time between mutations at a single site in the genome? The distribution below shows that often the waiting time is very short, but sometimes it is very long. There is a long tail to the exponential distribution. To think about why this is consider the relationship of the exponential to the binomial distribution earlier (random trials with success `p`). It only takes one success to end a trial, but sometimes you can have many many many trials occur in a row without a successful event happening. These rare runs of failures create the long tail of the exponential distrubution. 

In [ ]:
# random sample: waiting time is 1/x where x is the probability of an event
arr = np.random.exponential(scale=1/1e-8, size=100000)

# let's divide by 1e6 to get result in units of millions
arr = arr / 1e6

# bin the data
hist, edges = np.histogram(arr, bins=20, density=True)

# draw the bins
toyplot.bars((hist, edges), height=200, width=400, 
             label="Exponential distribution",
             xlabel="N trials until success",
             ylabel="Frequency");

# on average, it takes about 100M generations for a mutation to occur at a site
# when mutation rate is 1e-8 mutations per site per generation. This makes sense
# since 1e-8 means 1 / 10000000.
arr.mean().astype(int)

## Multivariate normal distribution

The multivariate normal distribution is a structured distribution in which a `covariance matrix`(shared variance) describes the variance in draws from a distribution as well as the *correlation* among values. This type of distribution is used commonly in biology, especially in the phylogenetics and population genetics. Using a `covariance matrix` we can represent the `evolutionary relationships` among populations or species (their shared ancestry) and thus model how similar they are expected to be under a null evolutionary model. In other words, it is a way of modeling the non-independece of species as data points (close relatives are expected to have more similar traits than more distant relatives due to their more recent common ancestry).  

Here we can demonstrate this phenomenon by drawing values from a normal distribution for three different species with different trait means (`[0, 5, 10]`), but dictate that there is a  correlation structure among them. Between the first species and the second species the correlation is high (covariance=0.75) while between the first and third species or the second and third it is low (covariance=0.15). A phylogenetic tree is drawn to show what this covariance structure would look like for three species. 

In [ ]:
# mean trait values 
mean = np.array([0, 5, 10])

# covariance structure (phylogeny) for three taxa
cov = np.array([
    [1.00, 0.75, 0.15],
    [0.75, 1.00, 0.15],
    [0.15, 0.15, 1.00],
])

# tree representation of same covariance structure
#
#     ----------+ 2
#     +
# -----
#     +     ----+ 1
#     ------+
#           ----+ 0
#

In [ ]:
# draw values from a MVN (normal distribution with covariance structure)
arr = np.random.multivariate_normal(mean, cov, 150)

As you can see in the first plot below, we generated a random distribution of points for each species over 150 replicates, where each replicate draws a mean trait value for each species. When we look at the data in one dimension it simply looks like three normal distributions of mean trait values drawn across many replicates, but when we compare the distributions in two dimensions we see there is a correlation structure: when the trait mean of species 0 is higher it is also higher in species 1. There is almost no correlation, however, between species 0 and 2 or species 1 and 2 trait means. 

Thus, the multivariate normal distribution was able to model the expected correlation structure among species with different levels of relatedness when evolving by a random (Brownian) process.

In [ ]:
# plot in 1-dimension
canvas = toyplot.Canvas(height=200, width=400)
axes = canvas.cartesian(xlabel="trait value", ylabel="count")
m0 = axes.bars(np.histogram(arr[:, 0], bins=10));
m1 = axes.bars(np.histogram(arr[:, 1], bins=10));
m2 = axes.bars(np.histogram(arr[:, 2], bins=10));

In [ ]:
# plot pairwise scatterplots
canvas = toyplot.Canvas(height=300, width=300)
axes = canvas.cartesian(xlabel="mean trait value spp x", ylabel="mean trait value spp y")
m0 = axes.scatterplot(arr[:, 0], arr[:, 1]);
m1 = axes.scatterplot(arr[:, 0], arr[:, 2]);
m2 = axes.scatterplot(arr[:, 1], arr[:, 2]);
canvas.legend([
    ("species 0 x 1", m0), 
    ("species 0 x 2", m1), 
    ("species 1 x 2", m2)],
    corner=('bottom-right', 50, 100, 50));

## Challenges
<div class="alert alert-success">
    At this point it is OK if these statistical distributions are very new to you and you do not yet fully understand them. In fact, this is one of the great strengths of using the `np.random` module, it can help you to understand and grasp statistical concepts through practice and hands-on exercises. 
<br><br>
Please try to complete the challenges below to test your assessment. There is no requirement to submit this assignment.
</div>

In [ ]:
# sample ten random integers in the range 0-100


In [ ]:
# sample ten random floats in the range 0-100


In [ ]:
# sample 100 values from a normal distribution with mean 10 and stdev 2


In [ ]:
# calculate and print the mean and std of the array generated above


In [ ]:
# create a boolean mask of size 100 with 10 True values and 90 False values


In [ ]:
# create a boolean mask where each element is randomly drawn True with p=0.5


In [ ]:
# apply the boolean mask to an array of normally distributed values to subselect elements.
